In [92]:
import jieba # 非常流行的 Python 中文分词库
import os
from utils import word_seg 
from gensim.models import Word2Vec
from gensim.models.word2vec import PathLineSentences
import multiprocessing # 该模块提供了创建进程的功能，允许你在多个CPU核心上并行执行代码。

In [94]:
source_folder='./three_kingdoms/source'
segment_folder = './three_kingdoms/segment'

word_seg.segment_folder_texts(source_folder, segment_folder)

In [95]:
sentences = PathLineSentences(segment_folder)

model2 = Word2Vec(
    sentences=sentences,
    vector_size=128,
    window=8,
    min_count=3,
    sg=1,
    negative=15,
    sample=1e-5,          # 高频词下采样，抑制“之/也/曰”等，但不硬删
    epochs=15,
    workers=multiprocessing.cpu_count(),
    seed=42
)

model2.save('./models/word2Vec.model')


In [96]:
def show_similar(model: Word2Vec, word: str, topn: int = 10) -> None:
    if word in model.wv.key_to_index:
        print(f'与「{word}」最相近的 {topn} 个词：')
        for w, s in model.wv.most_similar(word, topn=topn):
            print(f'  {w}\t{s:.3f}')
    else:
        print(f'⚠️ 词「{word}」不在词表（可能频次低于 min_count）。')

show_similar(model2, "曹操", topn=10)

与「曹操」最相近的 10 个词：
  程昱	0.989
  使人	0.988
  陈登	0.987
  切不可	0.987
  同往	0.987
  杨松	0.987
  袁绍	0.986
  必	0.986
  布乃	0.986
  依言	0.986


In [97]:
model2.wv.most_similar(positive=["曹操","刘备"], negative=["张飞"], topn=10)


[('天下', 0.957622230052948),
 ('臣', 0.9545016288757324),
 ('陛下', 0.9515909552574158),
 ('朕', 0.9498542547225952),
 ('先帝', 0.9491122364997864),
 ('卿', 0.9490923881530762),
 ('则', 0.9480808973312378),
 ('汉室', 0.9474281072616577),
 ('托孤', 0.9433239102363586),
 ('兴兵', 0.9432362914085388)]